In [42]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

# Load the trained model
loaded_model = keras.models.load_model('derment_ai.keras')

def enhance_image(image):
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Convert to YUV color space
    yuv = cv2.cvtColor(blurred, cv2.COLOR_BGR2YUV)
    
    # Apply Laplacian filter for edge enhancement on Y channel
    laplacian = cv2.Laplacian(yuv[:,:,0], cv2.CV_64F)
    
    # Normalize Laplacian for better contrast
    laplacian = cv2.normalize(laplacian, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    # Combine Laplacian with original Y channel to enhance edges
    enhanced_y = cv2.addWeighted(yuv[:,:,0], 1.5, laplacian, -0.5, 0)
    
    # Apply histogram equalization to enhance contrast
    enhanced_y = cv2.equalizeHist(enhanced_y)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for better local contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced_y = clahe.apply(enhanced_y)
    
    # Apply unsharp masking for edge and detail enhancement
    blurred = cv2.GaussianBlur(image, (0,0), 3)
    unsharp_mask = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)
    
    # Merge enhanced Y channel with original U and V channels
    enhanced_yuv = cv2.merge([enhanced_y, yuv[:,:,1], yuv[:,:,2]])
    
    # Convert back to BGR color space
    enhanced_bgr = cv2.cvtColor(enhanced_yuv, cv2.COLOR_YUV2BGR)
    
    return enhanced_bgr

# Function to preprocess a single image
def preprocess_image(image):
    if isinstance(image, str):
        image = cv2.imread(image)
        if image is None:
            print("Error: Unable to read image")
            return None
    # Resize the image to match the input shape of the model (224x224)
    img_resized = cv2.resize(image, (224, 224))
    # Normalize the image
    img_scaled = img_resized / 255.0
    # Convert the image to a numpy array
    img_array = np.expand_dims(img_scaled, axis=0)
    return img_array

# Function to predict disease type from an image
def predict_disease(image):
    # Preprocess the image
    img_array = preprocess_image(image)
    if img_array is None:
        return None
    # Make prediction using the loaded model
    prediction = loaded_model.predict(img_array)
    # Convert prediction to disease label
    disease_labels = ['cellulitis', 'impetigo', 'athlete-foot', 'nail-fungus', 'ringworm', 
                      'cutaneous-larva-migrans', 'chickenpox', 'shingles']
    predicted_label = disease_labels[np.argmax(prediction)]
    return predicted_label

# Test on individual image
image_path = '/Users/varunkaleeswaranbalasubramani/Downloads/download (1).jpeg'
image = cv2.imread(image_path)

enhanced_image = enhance_image(image)
predicted_disease = predict_disease(enhanced_image)
if predicted_disease is not None:
    print("Predicted Disease:", predicted_disease)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Disease: impetigo
